In [42]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

In [44]:
uk_emb = KeyedVectors.load_word2vec_format("/Users/ivanpaseka/ML/cc.uk.300.vec")

In [45]:
ru_emb = KeyedVectors.load_word2vec_format("/Users/ivanpaseka/ML/cc.ru.300.vec")

In [48]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240028262138367),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401415944099426)]

In [50]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999998807907104),
 ('липень', 0.9096440076828003),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992519974708557),
 ('жовтень', 0.8810409307479858),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592805862426758),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

In [52]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293080925941467),
 ('офор', 0.23256802558898926),
 ('deteydlya', 0.230317160487175),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608886659145355),
 ('подход', 0.2230587750673294),
 ('иболее', 0.22003725171089172),
 ('2015Александр', 0.21872763335704803),
 ('конструктивен', 0.21796567738056183)]

In [54]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf_8_sig') as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [56]:
uk_ru_train, X_train, Y_train = load_word_pairs("/Users/ivanpaseka/ML/ukr_rus.train.txt")

In [58]:
uk_ru_test, X_test, Y_test = load_word_pairs("/Users/ivanpaseka/ML/ukr_rus.test.txt")

In [60]:
from sklearn.linear_model import LinearRegression

# YOUR CODE HERE
mapping = LinearRegression().fit(X_train, Y_train)

In [62]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541592955589294),
 ('июнь', 0.8411962985992432),
 ('март', 0.8397400379180908),
 ('сентябрь', 0.8359214663505554),
 ('февраль', 0.8328748941421509),
 ('октябрь', 0.8311805725097656),
 ('ноябрь', 0.8278146982192993),
 ('июль', 0.823634922504425),
 ('август', 0.8120611906051636),
 ('декабрь', 0.8037999272346497)]

In [64]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE
        for (f, s) in ru_emb.most_similar(mapped_vectors[i].reshape(1, -1), topn=topn):
            if ru == f:
                num_matches += 1
    precision_val = num_matches / len(pairs)
    print(precision_val)
    return precision_val

In [66]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

0.0
1.0
1.0


In [68]:
assert precision(uk_ru_test, Y_test) == 1.0

1.0


In [70]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.811

0.6356589147286822
0.8113695090439277


In [72]:
from numpy.linalg import svd

def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOU CODE HERE
    u, s, vh = svd(np.matmul(X_train.T, Y_train), full_matrices=True)
    return np.matmul(u, vh)

In [74]:
W = learn_transform(X_train, Y_train)

In [76]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])[0]

('апрель', 0.8237907886505127)

In [78]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

0.6537467700258398
0.8242894056847545


In [82]:
with open("/Users/ivanpaseka/ML/fairy_tale.txt", "r", encoding='utf8') as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [88]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

def translate(sentence):
    """
    sentence: строка на украинском
    возвращает перевод (строка на русском)
    """
    tokens = tokenizer.tokenize(sentence.lower())
    translated_tokens = []

    for tok in tokens:
       
        if tok not in uk_emb:          
            translated_tokens.append(tok)
            continue

        uk_vec = uk_emb[tok]
        mapped_vec = np.matmul(uk_vec, W)
        ru_word = ru_emb.most_similar([mapped_vec], topn=1)[0][0]
        translated_tokens.append(ru_word)

    return " ".join(translated_tokens)

In [90]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [92]:
translate("кіт зловив мишу")

'кот поймал мышку'

In [94]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк – панібрат

src: як була собі лисичка та зробила хатку, та й живе. а це приходять холоди. от лисичка замерзла та й побігла в село вогню добувать, щоб витопити. прибігає до одної баби та й каже:
dst: как была себе лисичка и сделала избушку , и и живет . а оно приходят морозы . из лисичка замерзла и и побежала во село огня добувать , чтобы витопити . прибегает к одной бабы и и говорит :

src: — здорові були, бабусю! з неділею... позичте мені огню, я вам одслужу.
dst: — здоровые были , бабушку ! со воскресеньем ... позичте мне огня , мной тебе одслужу .

src: — добре, — каже, — лисичко - сестричко. сідай погрійся трохи, поки я пиріжечки повибираю з печі!
dst: — хорошо , — говорит , — лисичко – сестричка . садись погрійся немного , пока мной пирожки повибираю со печи !

src: а баба макові пиріжки пекла. от баба вибирає пиріжки та на столі кладе, щоб прохололи; а лисичка підгляділа та за пиріг, та з хати... виїла мач